# [Module 2.3] 세이지 메이커 사용자 도커 이미지로 인퍼런스

## 필수 사항
- 이 노트북은 sagemaker_inference_container/container-inference/ 의 아래 두개의 노트북을 먼저 실행해야 합니다.
    - 1.1.Build_Docker.ipynb
    - 2.1.Package_Model_Artifact.ipynb
본 워크샵의 모든 노트북은 `conda_python3` 추가 패키지를 설치하고 모두 이 커널 에서 작업 합니다.


## 이 노트북은 아래와 같은 작업을 합니다.

- 1. 환경 셋업
- 2. 로컬 모드에서 배포 및 추론 테스트
- 3. 세이지 메이커 (호스트 모드)에서 배포 및 추론 테스트
- 4. 세이지 메이커 신규 모델로 앤드포인트 업데이트
- 5. 신규 엔드포인트 생성
- 6. 신규 앤드포인트 생성 후에 Bake Time (예: 5분) 지난 후에 기존 엔드포인트 삭제
- 7. 마지막 신규 엔드포인트 리소스 제거


---    
    

# 1. 환경 셋업

## 기본 세팅
사용하는 패키지는 import 시점에 다시 재로딩 합니다.

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

이전 노트북에서 인퍼런스 테스트를 완료한 티펙트를 가져옵니다.
- inference_docker_image: 사용자 정의 도커 이미지
- byom_artifact: 추론 코드를 포함한 모델 아티펙트

In [4]:
%store -r inference_docker_image
# %store -r artifact_path
%store -r byom_artifact
%store -r bucket
%store -r prefix
%store -r artifact_path

In [5]:
print("inference_docker_image: \n", inference_docker_image)
print("byom_artifact: \n", byom_artifact)
print("artifact_path: \n", artifact_path)

inference_docker_image: 
 057716757052.dkr.ecr.us-east-1.amazonaws.com/ncf-sagemaker-inference
byom_artifact: 
 s3://sagemaker-us-east-1-057716757052/sm2fraud/new2train2model/model.tar.gz
artifact_path: 
 s3://sagemaker-us-east-1-057716757052/pytorch-training-2022-10-14-13-19-13-618/output/model.tar.gz


In [27]:
# 파이프라인 생성
# inference_docker_image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3'
# byom_artifact = 's3://sagemaker-us-east-1-057716757052/ncf/repackage/model/2022-11-14-12-49-10/model.tar.gz'

inference_docker_image = '763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-inference:1.8.1-gpu-py3'
byom_artifact = 's3://sagemaker-us-east-1-057716757052/ncf/repackage/model/2022-11-16-09-12-31/model.tar.gz'


In [28]:
import sagemaker

role = sagemaker.get_execution_role()

# 2. 로컬 모드에서 배포 및 추론 테스트


## Python SDK를 통한 세이지 메이커 모델 생성
- 로컬 엔드포인트에 적재될 세이지 메이커 모델을 생성홥니다.
- 세이지 메이커 모델은 크게 아래 두가지 요소가 있습니다.
    - 인퍼런스 이미지 컨테이너 위치
    - 모델 아티펙트 위치

In [29]:
from datetime import datetime
suffix = f"{datetime.today().strftime('%Y-%m-%d-%H-%M-%S')}"

# 로컬 노트북 인스턴스에서 사용할 로컬 세션
local_session = sagemaker.local.LocalSession()
local_session.config = {'local' : {'local_code':True}}
instance_type = 'local_gpu'
local_model_name = 'sm-local-model-' + str(suffix)
local_endpoint_name = 'local-endpoint-' + str(suffix)
print("local_endpoint_name: ", local_endpoint_name)

local_endpoint_name:  local-endpoint-2022-11-16-09-26-03


### 로컬 세이지 메이커 모델 정의

In [30]:
from sagemaker.model import Model


def create_sm_model(ecr_image, model_artifact,model_name,  role, session):
    model = Model(
                name = model_name,
                image_uri = ecr_image,
                model_data = model_artifact,
                role=role,
                sagemaker_session= session,
                 )
    print("model: ", model)
    return model

local_model = create_sm_model(inference_docker_image, byom_artifact, 
                              local_model_name, role, 
                              local_session)    

model:  <sagemaker.model.Model object at 0x7ff8266887c0>


### 로컬 엔드포인트 생성

- instance_type=='local' 시에는 최초 실행시에 인퍼런스 이미지를 다운로드로 하는 약 3분 걸리고, 이후에는 바로 실행이 됩니다.

In [31]:
%%time

local_predictor = local_model.deploy(initial_instance_count=1, instance_type=instance_type,
                         endpoint_name = local_endpoint_name,                                   
                        )

Attaching to takz4esic5-algo-1-whw8e
takz4esic5-algo-1-whw8e | Collecting nvidia-ml-py3==7.352
takz4esic5-algo-1-whw8e |   Downloading nvidia-ml-py3-7.352.0.tar.gz (19 kB)
takz4esic5-algo-1-whw8e |   Preparing metadata (setup.py) ... done
takz4esic5-algo-1-whw8e | Collecting pandas==0.24.2
takz4esic5-algo-1-whw8e |   Downloading pandas-0.24.2-cp36-cp36m-manylinux1_x86_64.whl (10.1 MB)
     |████████████████████████████████| 10.1 MB 40.5 MB/s            
takz4esic5-algo-1-whw8e | Collecting numpy==1.16.6
takz4esic5-algo-1-whw8e |   Downloading numpy-1.16.6-cp36-cp36m-manylinux1_x86_64.whl (17.4 MB)
     |████████████████████████████████| 17.4 MB 70.2 MB/s            
takz4esic5-algo-1-whw8e | Requirement already satisfied: torch==1.8.1 in /opt/conda/lib/python3.6/site-packages (from -r /opt/ml/model/code/requirements.txt (line 4)) (1.8.1)
takz4esic5-algo-1-whw8e | Collecting gensim==3.7.1
takz4esic5-algo-1-whw8e |   Downloading gensim-3.7.1-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
   

## 테스트 데이터 세트 로딩
- 로컬에서 저장된 데이터를 가져와서 데이터를 변환 합니다.
- batch_size 만큼 데이터를 로딩하는 데이터 로더를 정의 합니다.

In [32]:
import data_utils 
train_data, test_data, user_num ,item_num, train_mat = data_utils.load_all(test_num=100)

##  추론을 위한  데이터 세트 로딩
- 전부 데이터를 로딩할 필요가 없지만, 여기서는 기존에 사용한 함수를 이용하기 위해서 전체 데이터를 로드 합니다. 
    - 실제 데이터로 구현시에는 따로이 로드 함수를 사용하시기를 권장 합니다.


In [33]:
class Params:
    def __init__(self):
        # self.epochs = 1        
        self.num_ng = 4
        self.batch_size = 256
        self.test_num_ng = 99
        self.factor_num = 32
        self.num_layers = 3
        self.dropout = 0.0
        # self.lr = 0.001
        self.top_k = 10
        self.out = True
        # self.gpu = "0"
                        
args = Params()
print("# of batch_size: ", args.batch_size)


import torch.utils.data as data

test_dataset = data_utils.NCFData(
		test_data, item_num, train_mat, 0, False)

test_loader = data.DataLoader(test_dataset,
		batch_size=args.test_num_ng+1, shuffle=False, num_workers=0)



# of batch_size:  256


## 추론할 Paylaod 하나를 생성

In [34]:
for user, item, label in test_loader:   
    user_np = user.detach().cpu().numpy()
    item_np = item.detach().cpu().numpy()            
    break
payload = {'user':user_np.tolist(), 'item':item_np.tolist()}
print("payload: ", payload)


payload:  {'user': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'item': [25, 1064, 174, 2791, 3373, 269, 2678, 1902, 3641, 1216, 915, 3672, 2803, 2344, 986, 3217, 2824, 2598, 464, 2340, 1952, 1855, 1353, 1547, 3487, 3293, 1541, 2414, 2728, 340, 1421, 1963, 2545, 972, 487, 3463, 2727, 1135, 3135, 128, 175, 2423, 1974, 2515, 3278, 3079, 1527, 2182, 1018, 2800, 1830, 1539, 617, 247, 3448, 1699, 1420, 2487, 198, 811, 1010, 1423, 2840, 1770, 881, 1913, 1803, 1734, 3326, 1617, 224, 3352, 1869, 1182, 1331, 336, 2517, 1721, 3512, 3656, 273, 1026, 1991, 2190, 998, 3386, 3369, 185, 2822, 864, 2854, 3067, 58, 2551, 2333, 2688, 3703, 1300, 1924, 3118]}


## 엔드 포인트 추론
- Boto3 invoke_endpoint() 로 추론

In [35]:
import boto3

if instance_type == 'local_gpu':
    runtime_client = sagemaker.local.LocalSagemakerRuntimeClient()    
    endpoint_name = local_endpoint_name
else:
    runtime_client = boto3.Session().client('sagemaker-runtime')



### [중요] JSON type 의 payload 를 String 으로 직렬화 해서 제공 함.
```python
payload_dump = json.dumps(payload)
```

In [36]:
import json, time
from inference_utils import invoke_endpoint
payload_dump = json.dumps(payload)

start_time = time.time()
result = invoke_endpoint(runtime_client, endpoint_name, 
                         payload_dump,
                         content_type='application/json'
                        )

print("--- %s seconds ---" % (time.time() - start_time))
print('result: ', result)

takz4esic5-algo-1-whw8e | 2022-11-16 09:27:14,213 [INFO ] W-9001-model_1-stdout MODEL_LOG - #### input_fn starting ######
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:14,213 [INFO ] W-9001-model_1-stdout MODEL_LOG - content_type: application/json
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:14,213 [INFO ] W-9001-model_1-stdout MODEL_LOG - #### type of input data: <class 'str'>
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:14,214 [INFO ] W-9001-model_1-stdout MODEL_LOG - #### predict_fn starting ######
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:14,214 [INFO ] W-9001-model_1-stdout MODEL_LOG - #### type of input data: <class 'list'>
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:15,694 [INFO ] W-9001-model_1-stdout MODEL_LOG - recommends:  [128, 25, 273, 174, 58, 175, 617, 1539, 1018, 1902]
takz4esic5-algo-1-whw8e | 2022-11-16 09:27:15,695 [INFO ] W-9001-model_1-stdout MODEL_METRICS - PredictionTime.Milliseconds:1481.6|#ModelName:model,Level:Model|#hostname:58871a9081a1,requestID:bfe0f974-8c57-4a99

## 로컬 엔드 포인트 삭제
- 기존에 생성한 세이지 메이커 모델, 앤드포인트 컨피그, 앤드포인트 삭제

In [37]:
from inference_utils import delete_endpoint

client = sagemaker.local.LocalSagemakerClient()
delete_endpoint(client, endpoint_name)

Gracefully stopping... (press Ctrl+C again to force)
--- Deleted model: sm-local-model-2022-11-16-09-26-03
--- Deleted endpoint: local-endpoint-2022-11-16-09-26-03
--- Deleted endpoint_config: local-endpoint-2022-11-16-09-26-03
